In [1]:
from ris import pysqldb
from IPython.display import clear_output
from collections import defaultdict
from collections import defaultdict, namedtuple
import datetime 
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

print 'Notebook run: {}'.format(timestamp)
print os.getcwd()

Notebook run: 2020-02-06_18:32
E:\RIS\Staff Folders\Samuel\Requests\SH\Hudson_SQ


In [2]:
#Database connection
cdb = pysqldb.DbConnect(server='DOTDEVRHPGSQL01', database='CRASHDATA', type='PG')


Additional database connection details required:
User name (crashdata):soge
Password (crashdata)········
Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-02-06 18:32:25


### Writing Hudson Square Study Area Nodes to PG

In [3]:
cdb.csv_to_table(input_file='E:\RIS\Staff Folders\Samuel\Requests\SH\Hudson_SQ\Hudson_Sq.csv',schema='working', table_name = 'hsq', overwrite=True)

Reading data into Database



377it [00:09, 40.49it/s]



377 rows added to working.hsq



In [4]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-02-06 18:32:49


### Creating table for all crash data

In [5]:
cdb.query("""

DROP TABLE IF EXISTS working.hudson_sq; 

CREATE TABLE working.hudson_sq AS 

SELECT distinct nys_a.*
FROM archive."2019_11_13_nysdot_all" nys_a
JOIN archive."18d.2019-11-13_node" lion
on nys_a.nodeid::int = lion.nodeid::int
WHERE nys_a.case_yr>= 2013 and nys_a.case_yr<=2017
and nys_a.nodeid::int in(SELECT nodeid 
FROM working.hsq);

                        
GRANT ALL on working.hudson_sq to public;""")

Query run in 24 seconds


In [6]:
cdb.connect()

Database connection (PG) to CRASHDATA on DOTDEVRHPGSQL01 - user: soge 
Connection established 2020-02-06 18:33:14


### Collecting ped inj, bike iInj, mvo inj, total inj, severe injuries,	and ksi	data by masterid/injury rank


In [7]:
hudson_sq = cdb.dfquery("""


SELECT Dense_Rank() OVER (ORDER BY x."KSI" DESC) "KSI Ranking"
      ,Dense_Rank() OVER (ORDER BY x."All Inj" DESC) "Inj Ranking" 
      ,x.*
FROM(   SELECT inj.*
          , coalesce(fat."Fatalities",0) "Fatalities"
          , inj."Severe Injuries" + coalesce(fat."Fatalities",0) "KSI" FROM (
    --Hudson Square Injuries-----------------------
    SELECT  masterid
           ,coalesce(sum(CASE WHEN accd_type_int = 1 THEN num_of_inj END),0) "Ped Inj"
           ,coalesce(sum(CASE WHEN accd_type_int = 2 THEN num_of_inj END),0) "Bike Inj"
           ,coalesce(sum(CASE WHEN accd_type_int = 3 THEN num_of_inj END),0) "Mvo Inj"
           ,coalesce(sum(num_of_inj),0) "All Inj"
           ,coalesce(sum(length(ext_of_inj::text) - length(replace(ext_of_inj::text, 'A'::text, ''::text))),0) as "Severe Injuries"
    FROM working.hudson_sq
    GROUP BY  masterid
    ) inj
    LEFT JOIN (--Hudson Square Fatalities---------------------
           SELECT masterid, count(case when date_part('year',acdate) between 2013 and 2017 then id_ end) as "Fatalities"
           FROM public.fatality_nycdot_current fat
           WHERE  fat.nodeid::int in (select nodeid::int from working.hudson_sq)
           GROUP BY masterid) fat
    on inj.masterid = fat.masterid) x



""")  

### Street names for each masterid 

In [8]:
st_names = cdb.dfquery("""  

select masterid::int, array_agg(street) "Street Names" 
from(select distinct masteridto masterid, street
     from lion 
     where masteridto::int in (select masterid::int from working.hudson_sq)

     union

     select distinct masteridfrom masterid, street
     from lion 
     where masteridfrom::int in (select masterid::int from working.hudson_sq)
) st_names
group by masterid""" )

In [9]:
hsq = hudson_sq.merge(st_names, how = 'left', on='masterid')

In [10]:
hsq.to_csv('Hudson_Square_Ranking.csv',index = False)